Lesson 12 Chapter 5 LLMを活用したExcel操作の自動化手法

In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

Lesson 12 Chapter 5.1 欠損データの補完

In [2]:
# 住所から県名を推測して補完する関数
def extract_prefecture(address):
    # プロンプトの作成
    prompt_text = f"次の住所に基づいて、対応する都道府県名を推測し、「都道府県名」のみを答えてください：{address}"
    
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # キーワード抽出結果を取得
        keywords = response.choices[0].message.content.strip()
        return keywords
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [3]:
# テストコードで確認
text = extract_prefecture("渋谷区道玄坂1丁目16-3")
print(text)

東京都


In [4]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='住所')

# A列の各行のアンケートに対してキーワード抽出を実行し、B列に書き込む
df['都道府県'] = df['住所'].apply(extract_prefecture)

# 結果をExcelファイルに保存
df.to_excel("都道府県付住所.xlsx", index=False)

print("Excelファイルに結果を保存しました。")

処理を開始します。
Excelファイルに結果を保存しました。


Lesson 12 Chapter 5.2 データの整形

In [5]:
# 電話番号のフォーマットを統一する関数
def format_phone_numbers(phone_number):
    # プロンプトの作成
    prompt_text = f"次の電話番号を統一した形式「xxx-xxxx-xxxx」に変換してください。電話番号のみを出力してください: {phone_number}"
    
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # キーワード抽出結果を取得
        keywords = response.choices[0].message.content.strip()
        return keywords
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [6]:
text = format_phone_numbers("080.1234.5678")
print(text)

080-1234-5678


In [7]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='電話番号')

# A列の各行のアンケートに対してキーワード抽出を実行し、B列に書き込む
df['フォーマット後'] = df['電話番号'].apply(format_phone_numbers)

# 結果をExcelファイルに保存
df.to_excel("フォーマット後電話番号.xlsx", index=False)

print("Excelファイルに結果を保存しました。")

処理を開始します。
Excelファイルに結果を保存しました。


Lesson 12 Chapter 5.3 表記の揺れの統一（ツール呼出の応用）

In [8]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
  ChatCompletionToolParam({
    "type": "function",
    "function": {
      "name": "get_products",
      "description": "グループごとに統一した商品名「group_name」と、グループに属する商品名のリスト「products」を結合した結果を返す",
      "parameters": {
        "type": "object",
        "properties": {
          "product_data": {
            "type": "array",
            "items": {
              "product_name": {
                "type": "string",
                "description": "グループごとに統一した商品名",
              },
              "products": {
                "type": "string",
                "description": "グループに属する商品名をカンマ区切りで並べた文字列",
              },
            },
          }
        },
        "required": ["product_name", "products"],
      },
    },
  })
]

In [9]:
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='商品名')

# 2. データをLLM用にテキスト形式に変換
product_data_text = df.astype(str)
prompt_text = f"""
次の商品名のリストから、似た名前の商品をグループ化し、グループごとに統一した商品名「product_name」を提案し、グループに属する商品名をカンマ区切りで並べた文字列「products」を作成し、「product_nameとproductsのリスト」で構成されるJSONデータ「product_data」を出力してください。
{product_data_text}
"""

# 3. OpenAI APIの呼び出し
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_text},
    ],
    tools=tools,
    tool_choice={
        "type":"function",
        "function":{"name":"get_products"}
    },
    response_format={"type":"json_object"}
)

# 4. 分析結果をJSONオブジェクトに変換
# ツール呼出情報を取得
tool = response.choices[0].message.tool_calls[0]
# JSONオブジェクトに変換
data = json.loads(tool.function.arguments)

# データフレームに変換
df_out = pd.DataFrame(data['product_data'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("商品名グループ化結果.xlsx", index=False)

print("Excelファイルに処理結果を保存しました。")

処理を開始します。
Excelファイルに処理結果を保存しました。
